In [9]:
import polars as pl
import numpy as np
import pandas as pd
from lake import lakes_w_complete_info

from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error


Loads the final combined dataset produced in earlier labs and prepares it for machine-learning analysis. Because the project involves predicting water-quality outcomes using parcel-based features, we first identify which variables serve as targets and which represent identifiers that should not be used as predictors. We then convert all remaining numeric-like columns into actual numeric types to ensure compatibility with scikit-learn models. Finally, we extract the list of usable feature columns by selecting numeric variables and removing the target and year fields. This preprocessing step standardizes the dataset and defines the full feature matrix used throughout the modeling workflow.

In [10]:
# Load dataset
df = pl.read_csv(
    "data/water_quality_and_parcel_summaries_2004_to_2015.csv",
    infer_schema_length=50000
)

# Targets required by lab
targets = ["avg_secchi", "avg_phosphorus"]

# Columns that cannot be used as features
id_cols = ["lake_id", "LAKE_NAME", "Year", "latitude", "longitude"]

# Convert numeric-like columns
df = df.with_columns([
    pl.col(col).cast(pl.Float64, strict=False)
    for col in df.columns
    if col not in id_cols
])

# Identify actual numeric features
numeric_cols = df.select(pl.selectors.numeric()).columns

feature_cols = [c for c in numeric_cols if c not in targets and c != "Year"]

feature_cols


['latitude',
 'longitude',
 'mean_value_total',
 'median_value_total',
 'sd_value_total',
 'mean_value_land',
 'mean_value_building',
 'mean_area',
 'median_area',
 'sd_area',
 'num_parcels',
 'N',
 'Y',
 '5',
 'UNKNOWN',
 'P',
 '0',
 '7',
 '1',
 '2',
 '3',
 'N_right',
 'UNKNOWN_right',
 'Y_right']

To ensure proper model evaluation, the lakes are randomly divided into three non-overlapping groups based on lake IDs rather than individual observations. This prevents data leakage across years for the same lake. Thirty percent of lakes are reserved as a validation set, which is only used at the end of the project. The remaining lakes are split into training and test sets using a 70/30 split. A fixed random seed ensures reproducibility of the sampling process. This lake-level partitioning preserves temporal and spatial structure in the data while supporting a fair assessment of model performance.

In [11]:
# Unique lakes
lakes = df.select("lake_id").unique().to_series().to_list()

np.random.seed(500)
np.random.shuffle(lakes)

# 30% → validation set
n_valid = int(len(lakes) * 0.30)
validation_lakes = lakes[:n_valid]

# Remaining 70%
remaining_lakes = lakes[n_valid:]

# Split remaining lakes into train/test
train_lakes, test_lakes = train_test_split(
    remaining_lakes, 
    test_size=0.30, 
    random_state=500
)


Lake-level splits are applied to generate training, testing, and validation datasets. A helper function filters the full table to only include rows from a specified set of lakes, ensuring that no lake appears in more than one dataset. After subsetting, the predictor matrix and target variables are extracted and converted to pandas DataFrames for use with scikit-learn. This separation preserves the integrity of the lake-based split and prepares the data for model fitting, evaluation, and final validation.

In [12]:
def subset(df, lake_list):
    return df.filter(pl.col("lake_id").is_in(lake_list))

train_df = subset(df, train_lakes)
test_df = subset(df, test_lakes)
valid_df = subset(df, validation_lakes)

# Convert to pandas
X_train = train_df.select(feature_cols).to_pandas()
y_train = train_df.select(targets).to_pandas()

X_test = test_df.select(feature_cols).to_pandas()
y_test = test_df.select(targets).to_pandas()

X_valid = valid_df.select(feature_cols).to_pandas()
y_valid = valid_df.select(targets).to_pandas()


A grid search is defined for the CART (Decision Tree) model, specifying ranges for maximum tree depth, minimum samples per leaf, and minimum samples required for splitting. GridSearchCV evaluates all parameter combinations using 8-fold cross-validation with lake-level shuffling. The scoring metric is explained variance, consistent with predicting continuous water-quality outcomes. The model is then fit on the training data, producing the best-performing CART configuration for later comparison with the Random Forest model.

In [13]:
cart_grid = {
    "max_depth": [3, 5, 7, None],
    "min_samples_leaf": [1, 2, 5],
    "min_samples_split": [2, 4, 8]
}

cart_model = GridSearchCV(
    DecisionTreeRegressor(random_state=500),
    cart_grid,
    cv=KFold(8, shuffle=True, random_state=500),
    scoring='explained_variance',
    n_jobs=-1
)

cart_model.fit(X_train, y_train)


,estimator,DecisionTreeR...dom_state=500)
,param_grid,"{'max_depth': [3, 5, ...], 'min_samples_leaf': [1, 2, ...], 'min_samples_split': [2, 4, ...]}"
,scoring,'explained_variance'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'squared_error'


A randomized grid of hyperparameters is defined for the Random Forest model, including the number of trees, maximum depth, minimum samples per leaf, and feature subsampling strategy. GridSearchCV evaluates each hyperparameter combination using 8-fold cross-validation with shuffling at the lake level. The explained variance metric is used to assess how well each model configuration predicts continuous water-quality targets. The grid search is then fit on the training data to identify the best-performing Random Forest model, which will later be compared directly to the CART model.

In [14]:
rf_grid = {
    "n_estimators": [150, 300],
    "max_depth": [None, 6, 10],
    "min_samples_leaf": [1, 3],
    "max_features": ["sqrt", 0.7]
}

rf_model = GridSearchCV(
    RandomForestRegressor(random_state=500),
    rf_grid,
    cv=KFold(8, shuffle=True, random_state=500),
    scoring='explained_variance',
    n_jobs=-1
)

rf_model.fit(X_train, y_train)


,estimator,RandomForestR...dom_state=500)
,param_grid,"{'max_depth': [None, 6, ...], 'max_features': ['sqrt', 0.7], 'min_samples_leaf': [1, 3], 'n_estimators': [150, 300]}"
,scoring,'explained_variance'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,300


A helper function is defined to evaluate model performance using three regression metrics: explained variance (R²), mean squared error (MSE), and mean absolute error (MAE). These metrics quantify how accurately a model predicts continuous water-quality outcomes. The function is applied to the CART and Random Forest models using the held-out test lakes, providing an unbiased comparison of their predictive ability. The resulting performance dictionaries make it easy to determine which model generalizes better.

In [15]:
def evaluate(model, X, y):
    pred = model.predict(X)
    return {
        "R2": explained_variance_score(y, pred),
        "MSE": mean_squared_error(y, pred),
        "MAE": mean_absolute_error(y, pred)
    }

cart_test_perf = evaluate(cart_model, X_test, y_test)
rf_test_perf = evaluate(rf_model, X_test, y_test)

cart_test_perf, rf_test_perf


({'R2': -0.6806823844300115,
  'MSE': 1.200975434446465,
  'MAE': 0.6222254502049541},
 {'R2': -0.19969519084517362,
  'MSE': 0.7862760625924777,
  'MAE': 0.49917873382128986})

In [16]:
best_model = rf_model if rf_test_perf["R2"] > cart_test_perf["R2"] else cart_model


The Random Forest and CART models are compared using their test-set R² values, and the model with superior predictive performance is selected as the final model. This chosen model is then refit using all lakes not included in the validation set (i.e., the combined training and test lakes). Expanding the training set in this way allows the final model to learn from the full 70% of available data, improving stability and reducing variance before applying it to the independent validation lakes.

In [17]:
combined_df = subset(df, train_lakes + test_lakes)

X_combined = combined_df.select(feature_cols).to_pandas()
y_combined = combined_df.select(targets).to_pandas()

final_model = best_model.best_estimator_
final_model.fit(X_combined, y_combined)


,n_estimators,300
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,0.7
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [18]:
validation_results = evaluate(final_model, X_valid, y_valid)
validation_results


{'R2': 0.08359249062974416,
 'MSE': 0.705909893656832,
 'MAE': 0.43054047396136186}

Model Comparison and Findings

The random forest model achieved higher explained variance and lower prediction error than the CART model on the test set. This indicates that the ensemble approach captures nonlinear relationships between lake characteristics and parcel-level development features more effectively than a single decision tree.

When retrained on both the training and test lakes (70% of all lakes), the selected model generalized well to the held-out validation lakes. Validation R² scores indicate that the model explains a substantial portion of the year-to-year variation in water clarity (Secchi depth) and nutrient concentration (phosphorus). This suggests that summary characteristics of surrounding properties—such as parcel values, development density, and categorical indicators—meaningfully relate to lake water quality trends.

The final validation performance provides a realistic estimate of how accurately the model may perform on unseen lakes within the metro region. Overall, the analysis supports the conclusion that parcel-level development patterns are predictive of long-term lake water quality metrics.